This notebook walks through the level 1 analyses of a single subject for Developmental differences in learning from large losses study. Details of the behavioral analyses can be found [here](https://zenkavi.github.io/DevStudy_Analyses/output/reports/DevStudy_MG_Behavior.nb.html).


Import packages

In [1]:
import glob
import nibabel as nib
from nistats.first_level_model import FirstLevelModel
import numpy as np
import os
import pandas as pd
import pickle
import re
import sys
sys.path.append(os.environ['SERVER_SCRIPTS'])
from utils.mni2vox import mni2vox
import warnings

Import custom functions

In [2]:
from level_1_utils import make_contrasts, add_transform, stdize, get_conditions, get_confounds

Import plotting packages

In [3]:
from matplotlib.colors import ListedColormap
import matplotlib.patheffects as PathEffects
import matplotlib.pyplot as plt
from scipy.stats import norm
import seaborn as sns
from nistats.reporting import plot_design_matrix, plot_contrast_matrix
import nilearn.plotting

%matplotlib inline

# Testing level 1 analysis script

Declare arguments

In [4]:
subnum = '100003'
pe = True
ev = True
data_loc = os.environ['DATA_LOC']
server_scripts = os.environ['SERVER_SCRIPTS']

Read in events files, make output directories and read in prediction errors

In [5]:
events_files = glob.glob('%s/sub-*/func/sub-*_task-machinegame_run-*_events.tsv'%(data_loc))
events_files.sort()

out_path = "%s/derivatives/nistats/level_1/sub-%s"%(data_loc,subnum)
if not os.path.exists(out_path):
    os.mkdir(out_path)

contrasts_path = "%s/contrasts"%(out_path)
if not os.path.exists(contrasts_path):
    os.mkdir(contrasts_path)

all_events = pd.DataFrame()

for cur_ef in events_files:
    df = pd.read_csv(cur_ef, sep = '\t')
    all_events = all_events.append(df, ignore_index= True)

all_events = all_events[all_events['response_time'].notnull()]
all_events.response_time = all_events.response_time/1000
mean_rt = all_events.response_time.mean()

del all_events

sub_events = [x for x in events_files if subnum in x]

all_pes = pd.read_csv('%s/rpe_cors/pred_rpes/exp_exp.csv'%(server_scripts))
sub_pes = all_pes.query('sub_id == @subnum')
del all_pes

all_evs = pd.read_csv('%s/rpe_cors/pred_evs/exp_exp.csv'%(server_scripts))
sub_evs = all_evs.query('sub_id == @subnum')
del all_evs

In [7]:
run_events = sub_events[0]
runnum = re.findall('\d+', os.path.basename(run_events))[1]

#fmri_img: path to preproc_bold that the model will be fit on
fmri_img = os.path.join(data_loc,"derivatives/fmriprep_1.4.0/fmriprep/sub-%s/func/sub-%s_task-machinegame_run-%s_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz"%(subnum, subnum, runnum))

#read in preproc_bold for that run
cur_img = nib.load(fmri_img)
cur_img_tr = cur_img.header['pixdim'][4]

#read in events.tsv for that run
cur_events = pd.read_csv(run_events, sep = '\t')
formatted_events = get_conditions(cur_events, runnum, mean_rt, sub_pes, pe, sub_evs, ev)

#process confounds
#['X','Y','Z','RotX','RotY','RotY','<-firsttemporalderivative','stdDVARs','FD','scrub']
cur_confounds = pd.read_csv(os.path.join(data_loc,"derivatives/fmriprep_1.4.0/fmriprep/sub-%s/func/sub-%s_task-machinegame_run-%s_desc-confounds_regressors.tsv"%(subnum, subnum, runnum)), sep='\t')
formatted_confounds = get_confounds(cur_confounds)

#define GLM parmeters
fmri_glm = FirstLevelModel(t_r=cur_img_tr,
                       noise_model='ar1',
                       standardize=False,
                       hrf_model='spm + derivative',
                       drift_model='cosine',
                       smoothing_fwhm=5,
                       mask='%s/derivatives/fmriprep_1.4.0/fmriprep/sub-%s/func/sub-%s_task-machinegame_run-%s_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz'%(data_loc, subnum, subnum, runnum))

/oak/stanford/groups/russpold/users/zenkavi/DevStudy_ServerScripts/nistats/level_1/level_1_utils.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  cond_m1_ev['EV'] = cond_m1_ev['EV'].sub(cond_m1_ev['EV'].mean())
/oak/stanford/groups/russpold/users/zenkavi/DevStudy_ServerScripts/nistats/level_1/level_1_utils.py:158: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  formatted_events = pd.concat([cond_m1_ev, cond_m2_ev, cond_m3_ev, cond_m4_ev, cond_m1_rt, cond_m2_rt, cond_m3_rt, cond_m4_rt, cond_hpe, cond_lpe, cond_junk], ignore_index=True)
/oak/stanford/

In [21]:
max_X = int(runnum)*30
run_evs = sub_evs.query('X<@max_X')
cond_ev = cur_events.query('trial_type == "stim_presentation"')
cond_ev = pd.concat([cond_ev.reset_index(drop=True), run_evs['EV'].reset_index(drop=True)], axis=1)
cond_ev    

,onset,duration,trial_type,response,stimulus,response_time,points_earned,iti_start_time,iti_length,rt_shift,EV
0,0.000,2.517,stim_presentation,NaN,2.0,NaN,NaN,551.782,0.519,0.627467,0.000000
1,5.046,3.642,stim_presentation,NaN,3.0,NaN,NaN,557.953,1.454,1.665467,0.000000
2,12.151,2.677,stim_presentation,NaN,1.0,NaN,NaN,564.093,1.381,0.054467,0.000000
3,18.218,2.613,stim_presentation,NaN,4.0,NaN,NaN,570.096,2.868,0.989467,0.000000
4,25.708,3.913,stim_presentation,NaN,4.0,NaN,NaN,578.886,3.060,-0.723533,0.000000
5,34.690,3.135,stim_presentation,NaN,4.0,NaN,NaN,587.090,0.942,-0.272533,-0.176820
6,40.777,2.727,stim_presentation,NaN,3.0,NaN,NaN,592.769,2.101,-0.189533,0.686947
7,47.614,2.912,stim_presentation,NaN,1.0,NaN,NaN,599.791,0.590,-0.368533,0.238462
8,53.125,2.576,stim_presentation,NaN,2.0,NaN,NaN,604.966,3.834,-0.245533,-0.075279
9,61.544,3.739,stim_presentation,NaN,1.0,NaN,NaN,614.548,1.607,0.669467,0.472318


In [ ]:
#fit glm to run image using run events
print("***********************************************")
print("Running GLM for sub-%s run-%s"%(subnum, runnum))
print("***********************************************")
fmri_glm = fmri_glm.fit(fmri_img, events = formatted_events, confounds = formatted_confounds)

In [ ]:
design_matrix = fmri_glm.design_matrices_[0]
contrasts = make_contrasts(design_matrix, pe)
contrasts['m1']

In [ ]:
z_map = fmri_glm.compute_contrast(contrasts["m1"], output_type='z_score')

In [ ]:
eff_map = fmri_glm.compute_contrast(contrasts["m1"], output_type='effect_size')

In [ ]:
eff_var_map = fmri_glm.compute_contrast(contrasts["m1"], output_type='effect_variance')

In [ ]:
stat_map = fmri_glm.compute_contrast(contrasts["m1"], output_type='stat')

In [ ]:
z_map_data = z_map.get_fdata()

In [ ]:
eff_map_data = eff_map.get_fdata()

In [ ]:
eff_var_map_data = eff_var_map.get_fdata()

In [ ]:
stat_map_data = stat_map.get_fdata()

In [ ]:
z_map_data.max()

In [ ]:
eff_map_data.max()

In [ ]:
stat_map_data.max()

In [ ]:
tmp1 = nib.load('/oak/stanford/groups/russpold/data/ds000054/0.0.4/derivatives/fsl/level_1/sub-400742/model1/run-001.feat/stats/cope1.nii.gz')

In [ ]:
tmp1.get_fdata().max()

In [ ]:
np.nanmax(eff_map_data/np.sqrt(eff_var_map_data))

In [ ]:
for run_events in sub_events:

    runnum = re.findall('\d+', os.path.basename(run_events))[1]

    exists = os.path.isfile(os.path.join(data_loc,"derivatives/fmriprep_1.4.0/fmriprep/sub-%s/func/sub-%s_task-machinegame_run-%s_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz"%(subnum, subnum, runnum)))

    if exists:

        #fmri_img: path to preproc_bold that the model will be fit on
        fmri_img = os.path.join(data_loc,"derivatives/fmriprep_1.4.0/fmriprep/sub-%s/func/sub-%s_task-machinegame_run-%s_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz"%(subnum, subnum, runnum))

        #read in preproc_bold for that run
        cur_img = nib.load(fmri_img)
        cur_img_tr = cur_img.header['pixdim'][4]

        #read in events.tsv for that run
        cur_events = pd.read_csv(run_events, sep = '\t')
        formatted_events = get_conditions(cur_events, runnum, mean_rt, sub_pes, pe)

        #process confounds
        #['X','Y','Z','RotX','RotY','RotY','<-firsttemporalderivative','stdDVARs','FD','scrub']
        cur_confounds = pd.read_csv(os.path.join(data_loc,"derivatives/fmriprep_1.4.0/fmriprep/sub-%s/func/sub-%s_task-machinegame_run-%s_desc-confounds_regressors.tsv"%(subnum, subnum, runnum)), sep='\t')
        formatted_confounds = get_confounds(cur_confounds)

        #define GLM parmeters
        fmri_glm = FirstLevelModel(t_r=cur_img_tr,
                               noise_model='ar1',
                               standardize=False,
                               hrf_model='spm + derivative',
                               drift_model='cosine',
                               smoothing_fwhm=5,
                               mask='%s/derivatives/fmriprep_1.4.0/fmriprep/sub-%s/func/sub-%s_task-machinegame_run-%s_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz'%(data_loc, subnum, subnum, runnum))

        #fit glm to run image using run events
        print("***********************************************")
        print("Running GLM for sub-%s run-%s"%(subnum, runnum))
        print("***********************************************")
        fmri_glm = fmri_glm.fit(fmri_img, events = formatted_events, confounds = formatted_confounds)

        print("***********************************************")
        print("Saving GLM for sub-%s run-%s"%(subnum, runnum))
        print("***********************************************")
        if pe:
            f = open('%s/sub-%s_run-%s_l1_%s_glm.pkl' %(out_path,subnum, runnum, 'pe'), 'wb')
        else:
            f = open('%s/sub-%s_run-%s_l1_glm.pkl' %(out_path,subnum, runnum), 'wb')
        pickle.dump(fmri_glm, f)
        f.close()

        #Save design matrix
        design_matrix = fmri_glm.design_matrices_[0]
        print("***********************************************")
        print("Saving design matrix for sub-%s run-%s"%(subnum, runnum))
        print("***********************************************")
        if pe:
            design_matrix.to_csv(os.path.join(out_path, 'sub-%s_run-%s_level1_%s_design_matrix.csv' %(subnum, runnum, 'pe')))
        else:
            design_matrix.to_csv(os.path.join(out_path, 'sub-%s_run-%s_level1_design_matrix.csv' %(subnum, runnum)))

        print("***********************************************")
        print("Running contrasts for sub-%s run-%s"%(subnum, runnum))
        print("***********************************************")
        contrasts = make_contrasts(design_matrix, pe)
        for index, (contrast_id, contrast_val) in enumerate(contrasts.items()):
            z_map = fmri_glm.compute_contrast(contrast_val, output_type='z_score')
            nib.save(z_map, '%s/sub-%s_run-%s_%s.nii.gz'%(contrasts_path, subnum, runnum, contrast_id))
        print("***********************************************")
        print("Done saving contrasts for sub-%s run-%s"%(subnum, runnum))
        print("***********************************************")

    else:
        print("***********************************************")
        print("No pre-processed BOLD found for sub-%s run-%s"%(subnum, runnum))
        print("***********************************************")

# Outputs of level 1 analyses

Let's look at the outputs  


## Design matrix
Design matrix for the last run

In [ ]:
plt.rcParams['figure.figsize'] = [10, 5]
plot_design_matrix(design_matrix)

Closer look at task regressors

In [ ]:
plot_design_matrix(design_matrix[['m1', 'm2', 'm3', 'm4','m1_rt', 'm2_rt', 'm3_rt', 'm4_rt','hpe', 'lpe']].reset_index(drop=True))

## Correlations between regressors

Correlations between regressors in the design matrix for the last run

In [ ]:
cc=np.corrcoef(design_matrix.T)
plt.imshow(cc,aspect='auto',interpolation='nearest', cmap=plt.cm.viridis)
plt.colorbar()

The highly correlated regressors are motion regressors

In [ ]:
design_matrix.columns[20:40]

## Contrast maps

Let's look at contrast maps for the last run

In [ ]:
zmap_paths = glob.glob('%s/*run-006*'%(contrasts_path))
zmap_paths.sort()

In [ ]:
for t_map in zmap_paths:
    nilearn.plotting.plot_glass_brain(nilearn.image.smooth_img(t_map, 8),
                                      display_mode='lyrz', colorbar=True, plot_abs=False, threshold=2.3, title=os.path.basename(t_map))

In [ ]:
for t_map in zmap_paths:
    nilearn.plotting.plot_stat_map(nilearn.image.smooth_img(t_map, 8), colorbar=True, threshold=2.3, title=os.path.basename(t_map))

For another run the contrast images look like:

In [ ]:
zmap_paths = glob.glob('%s/*run-003*'%(contrasts_path))
zmap_paths.sort()
for t_map in zmap_paths:
    nilearn.plotting.plot_stat_map(nilearn.image.smooth_img(t_map, 8), colorbar=True, threshold=2.3, title=os.path.basename(t_map))

# Extracting z-values 

## Manual extraction

In [ ]:
img_file_name = '%s/sub-%s_run-002_hpe.nii.gz'%(contrasts_path, subnum)
img = nib.load(img_file_name)
mask_file_name = '/oak/stanford/groups/russpold/data/ds000054/0.0.4/derivatives/rois/tpl-MNI152NLin2009cAsym_res-01_desc-brain_T1w/l_vstr_bin.nii.gz'
mask = nib.load(mask_file_name)

Extract ROI data manually as in: https://neurostars.org/t/extract-roi-voxels-using-nilearns-niftimasker/3977  

**Doesn't work because the BOLD does not have the same dimensionality**

In [ ]:
img_data = img.get_fdata()
mask_data = mask.get_fdata()
roi_nifti_data = np.where(mask_data == 1,img_data,0)
roi_nifti_img = image.new_img_like(img,roi_nifti_data)

In [ ]:
img_data.shape == mask_data.shape

Resample mask to statistical map  

Note: the resampled image is not longer binarized

In [ ]:
res_mask = nilearn.image.resample_to_img(mask, img)
res_mask_data = res_mask.get_fdata()
#binarize resampled mask data
res_mask_data = np.where(res_mask_data >0.1,1,0)
res_mask = nilearn.image.new_img_like(img, res_mask_data)

In [ ]:
nilearn.plotting.plot_stat_map(res_mask, colorbar=True, draw_cross=False)

Does it matter which contrast image the ROI is resampled to?

In [ ]:
img2_file_name = '/oak/stanford/groups/russpold/data/ds000054/0.0.4/derivatives/nistats/level_1/sub-407209/contrasts/sub-407209_run-002_hpe.nii.gz'
img2 = nib.load(img2_file_name)
res2_mask = nilearn.image.resample_to_img(mask, img2)
res2_mask_data = res2_mask.get_fdata()
#binarize resampled mask data
res2_mask_data = np.where(res2_mask_data >0.1,1,0)
res2_mask = nilearn.image.new_img_like(img2, res2_mask_data)
nilearn.plotting.plot_stat_map(res2_mask, colorbar=True, draw_cross=False)

No. The resampled mask seems the same regardless of what contrast image it is registered to

In [ ]:
(res_mask_data != res2_mask_data).sum()

In [ ]:
img_data = img.get_fdata()
roi_nifti_data = np.where(res_mask_data == 1,img_data,0)
roi_nifti_img = nilearn.image.new_img_like(img,roi_nifti_data)

In [ ]:
nilearn.plotting.plot_stat_map(roi_nifti_img, colorbar=True, draw_cross=False)

In [ ]:
roi_data = roi_nifti_data[roi_nifti_data != 0]
cur_betas = pd.DataFrame()
cur_betas["beta"] = roi_data


In [ ]:
models = glob.glob(os.path.join(server_scripts, 'rpe_cors/pred_rpes/*.csv'))
models = [os.path.splitext(os.path.basename(x))[0] for x in models]
model = models[7]
beta_img_paths = glob.glob('%s/derivatives/rpe_cors/%s/sub-*/contrasts/sub-*_run-*_*pe_betas.nii.gz'%(data_loc, model))
cur_beta_img = beta_img_paths[10]
os.path.basename(cur_beta_img).split("_")[2]

In [ ]:
nilearn.plotting.plot_stat_map(img, colorbar=True, threshold=2.3, draw_cross=False, cut_coords=[-11,13,-6])

In [ ]:
nilearn.plotting.plot_stat_map(img, colorbar=True, threshold=2.3, draw_cross=False, cut_coords=[-11,13,-6])

## Using freesurfer

Extract using freesurfer segmentation as described in https://groups.google.com/forum/#!topic/nipy-user/DjnN9q52vH8

**Not working due to broken freesurfer path**

In [ ]:
import nipype.interfaces.freesurfer as fs 
segstat = fs.SegStats() 
segstat.inputs.segmentation_file = mask_file_name 
segstat.inputs.in_file = img_file_name 
segstat.inputs.avgwf_file = True 
segstat.run() 

# Check dimensions of fmriprep output

In [ ]:
nib.load('/oak/stanford/groups/russpold/data/ds000054/0.0.4/derivatives/rois/tpl-MNI152NLin2009cAsym_res-01_desc-brain_T1w/tpl-MNI152NLin2009cAsym_res-01_desc-brain_T1w.nii.gz').shape

In [ ]:
nib.load(data_loc+'/derivatives/fmriprep_1.4.0/fmriprep/sub-400742/func/sub-400742_task-machinegame_run-005_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz').shape

In [ ]:
nib.load(data_loc+'/derivatives/fmriprep_1.4.0/fmriprep/sub-100003/func/sub-100003_task-machinegame_run-002_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz').shape

In [ ]:
nib.load(data_loc+'/derivatives/fmriprep_1.3.0/fmriprep/sub-400742/anat/sub-400742_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz').shape

In [ ]:
nib.load('/home/users/zenkavi/datalad/templateflow/tpl-MNI152NLin2009cAsym/tpl-MNI152NLin2009cAsym_res-02_desc-brain_T1w.nii.gz').shape

In [ ]:
tmp_path = '/oak/stanford/groups/russpold/users/ieisenbe/uh2/fmri_data/BIDS_data/derivatives/fmriprep/fmriprep/sub-s061/ses-1/func'
nib.load(tmp_path+'/sub-s061_ses-1_task-stopSignal_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz').shape

In [ ]:
tmp_path = '/oak/stanford/groups/russpold/data/ds000054/0.0.4/sub-400742/func'
nib.load(tmp_path+'/sub-400742_task-machinegame_run-001_bold.nii.gz').shape

In [ ]:
tmp_path = '/oak/stanford/groups/russpold/data/ds000054/0.0.4/sub-400742/anat'
nib.load(tmp_path+'/sub-400742_T1w.nii.gz').shape

In [ ]:
nib.load('/oak/stanford/groups/russpold/data/templateflow/tpl-MNI152NLin2009cAsym/tpl-MNI152NLin2009cAsym_res-01_desc-brain_T1w.nii.gz').shape